In [1]:
import pandas as pd
import vertica_python
import numpy as np
import sys
sys.path.append(r'C:\Users\danny.habetyes\Desktop\Code\Python\domo_api-master')
import domo_api
from datetime import datetime
import time

In [2]:
# Connection Info
conn_info = {'host': 'vertica.chotel.com',
             'port': '5433',
             'user': 'danny.habetyes',
             'password': 'Choice123',
             'database': 'choice_vertica_1',
             # autogenerated session label by default,
             'session_label': 'some_label',
             # default throw error on invalid UTF-8 results
             'unicode_error': 'strict',
             # SSL is disabled by default
             'ssl': False,
             # using server-side prepared statements is disabled by default
             'use_prepared_statements': False,
             # connection timeout is not enabled by default
             # 5 seconds timeout for a socket operation (Establishing a TCP connection or read/write operation)
             'connection_timeout': 150}

# Define function to pull dataframes
def vertica_pull(sql_string, emails=True, save = True, save_file="rename", file_type =".csv"):
    """
    emails: Rewrites first column header to be "Emails" for hashed list uploading
    save: defaults to True. Set to False to avoid writing CSV
    file_type: defaults to .csv
    """
    with vertica_python.connect(**conn_info) as connection:
        cur = connection.cursor()
        get = cur.execute(sql_string)
        df = pd.DataFrame(get.fetchall(), columns = [x[0] for x in get.description])
        if emails:
                df = df.rename(columns={df.columns[0]:"Email"})
        while save:
            df.to_csv(f'{save_file}{file_type}', index=False)
            return print(f'File saved as {save_file}{file_type}')
        return df

In [3]:
sql2 ="""
 WITH RESERVATIONS AS (
   select *
     from
   (
     select b.reservation_date
     , case when b.reservation_date between SYSDATE - 9 and SYSDATE -2 then '-7 Days'
     when b.reservation_date between (SYSDATE - 9)-364 and (SYSDATE -2)-364 then '-7 Days'
     when b.reservation_date between SYSDATE - 16 and SYSDATE -9 then '-14 Days'
     when b.reservation_date between (SYSDATE - 16)-364 and (SYSDATE -9)-364 then '-14 Days'
     when b.reservation_date between SYSDATE - 23 and SYSDATE -16 then '-21 Days'
     when b.reservation_date between (SYSDATE - 23)-364 and (SYSDATE -16)-364 then '-21 Days'
     else '-22 Days +' end as time_window
     , DATE_PART('MONTH', b.arrival_date) || '-' || DATE_PART('YEAR', b.arrival_date) as arrival_mo
     , DATE_PART('YEAR', b.reservation_date) as reservation_year
     , case when DATE_PART('YEAR', b.reservation_date) = '2020' then b.reservation_date
     else b.reservation_date+364 end as same_date
     , case when b.arrival_date - b.reservation_date <0 then 0
     else b.arrival_date - b.reservation_date end as AP_window   
     , case when b.arrival_date - b.reservation_date <0 then '0-1 Days'
     When b.arrival_date - b.reservation_date between 0 and 1 then '0-1 Days'
     When b.arrival_date - b.reservation_date between 2 and 3 then '2-3 Days'
     When b.arrival_date - b.reservation_date between 4 and 7 then '4-7 Days'
     When b.arrival_date - b.reservation_date between 8 and 14 then '8-14 Days'
     else '15+ Days' end as ABW  
     , b.conf_nbr
     , 1 as res_nbr                          
     , b.prop_city 
     , c.market_name 
     , p.mkt_location_des
     , b.prop_state
     , b.ta_client_id
     , i.iata_desc
     , i.media_desc           
     , row_number() over (partition by b.conf_nbr, b.ta_client_id, b.guest_city, b.prop_code order by b.reservation_date) as rn
     , b.prop_code
    
     from public.prd_reservations b
     inner join rm.nc_srp_grouping_detail r on b.srp_code = r.srp_code
     inner join sandbox.da_iata_lookup i on i.iata = b.ta_client_id
     inner join public.current_property p on p.prop = b.prop_code
     inner join prd_prop_info_detail c on b.prop_code = c.prop    
    
     where 1=1
     and i.media_desc in ('Meta','Search')
     and b.prop_country in ('US')
     and b.reservation_date > '04-01-2018'
     and b.canx_nbr = '0'
    
     group by b.reservation_date
     , case when b.reservation_date between SYSDATE - 16 and SYSDATE -2 then 'Y'
     when b.reservation_date between (SYSDATE - 16)-364 and (SYSDATE -2)-364 then 'Y'
     else 'N' end 
     , DATE_PART('MONTH', b.arrival_date) || '-' || DATE_PART('YEAR', b.arrival_date) 
     , DATE_PART('YEAR', b.reservation_date) 
     , case when DATE_PART('YEAR', b.reservation_date) = '2020' then b.reservation_date
     else b.reservation_date+364 end 
     ,  case when b.arrival_date - b.reservation_date <0 then 0
     else b.arrival_date - b.reservation_date end 
     , b.conf_nbr                 
     , b.prop_city
     ,c.market_name   
     , p.mkt_location_des
     , b.prop_state
     , b.ta_client_id
     , i.iata_desc
     , i.iata_desc
     , i.media_desc  
     , case when r.comission = 'Yes' then 'Y'
     when r.comission = 'No' then 'N'
     when r.srp_code like 'RACK%' then 'Y'
     when r.srp_code like '%GROUP%' then 'N'
     when r.srp_code like '%PKG%' then 'N'       
     else 'Y' end
     , case when b.arrival_date - b.reservation_date <0 then '0-1 Days'
     When b.arrival_date - b.reservation_date between 0 and 1 then '0-1 Days'
     When b.arrival_date - b.reservation_date between 2 and 3 then '2-3 Days'
     When b.arrival_date - b.reservation_date between 4 and 7 then '4-7 Days'
     When b.arrival_date - b.reservation_date between 8 and 14 then '8-14 Days'
     else '15+ Days' end                                          
     , b.guest_city
     , b.prop_code
    
     having case when r.comission = 'Yes' then 'Y'
     when r.comission = 'No' then 'N'
     when r.srp_code like 'RACK%' then 'Y'
     when r.srp_code like '%GROUP%' then 'N'
     when r.srp_code like '%PKG%' then 'N'       
     else 'Y' end = 'Y'          
   ) b
  
   where rn = 1
 ),


 REVENUE AS (
   select b.conf_nbr
   , round(sum(gross_resv),2) as gross_resv
   , round(sum(gross_nights),2) as gross_nights
   , round(sum(gross_revenue),2) as gross_revenue
   , round(sum(canx_resv),2) as canx_resv
   , round(sum(canx_nights),2) as canx_nights
   , round(sum(canx_revenue),2) as canx_revenue
   , round(sum(net_resv),2) as net_resv
   , round(sum(net_nights),2) as net_nights
   , round(sum(net_revenue),2) as net_revenue
  
   from public.prd_reservations b
   inner join rm.nc_srp_grouping_detail r on b.srp_code = r.srp_code
   inner join sandbox.da_iata_lookup i on i.iata = b.ta_client_id
   inner join public.current_property p on p.prop = b.prop_code   
  
   where 1=1
   and i.media_desc in ('Meta','Search')
   and b.prop_country in ('US')
   and b.reservation_date > '04-01-2018'
  
   group by b.conf_nbr                 
   , case when r.comission = 'Yes' then 'Y'
   when r.comission = 'No' then 'N'
   when r.srp_code like 'RACK%' then 'Y'
   when r.srp_code like '%GROUP%' then 'N'
   when r.srp_code like '%PKG%' then 'N'       
   else 'Y' end
  
   having case when r.comission = 'Yes' then 'Y'
   when r.comission = 'No' then 'N'
   when r.srp_code like 'RACK%' then 'Y'
   when r.srp_code like '%GROUP%' then 'N'
   when r.srp_code like '%PKG%' then 'N'       
   else 'Y' end = 'Y'          
 ),

 CANCELS AS (
   select b.conf_nbr
   , 1 as canx_nbr
  
   from public.prd_reservations b
   inner join rm.nc_srp_grouping_detail r on b.srp_code = r.srp_code
   inner join sandbox.da_iata_lookup i on i.iata = b.ta_client_id
   inner join public.current_property p on p.prop = b.prop_code   
  
   where 1=1
   and i.media_desc in ('Meta','Search')
   and b.prop_country in ('US')
   and b.reservation_date > '04-01-2018'
   and b.canx_nbr !='0'  
  
   group by b.conf_nbr                 
   , case when r.comission = 'Yes' then 'Y'
   when r.comission = 'No' then 'N'
   when r.srp_code like 'RACK%' then 'Y'
   when r.srp_code like '%GROUP%' then 'N'
   when r.srp_code like '%PKG%' then 'N'       
   else 'Y' end
  
   having case when r.comission = 'Yes' then 'Y'
   when r.comission = 'No' then 'N'
   when r.srp_code like 'RACK%' then 'Y'
   when r.srp_code like '%GROUP%' then 'N'
   when r.srp_code like '%PKG%' then 'N'       
   else 'Y' end = 'Y'   
 )

,
BEACH AS (
SELECT DISTINCT prop, count(*) as beach FROM prd_prop_amenity WHERE amnt_id = 'BCH' GROUP BY prop
),
HIKE AS (
SELECT DISTINCT prop, count(*) as hike FROM prd_prop_amenity WHERE amnt_id = 'HIKE' GROUP BY prop
),
JOG AS(
SELECT DISTINCT prop, count(*) as jog FROM prd_prop_amenity WHERE amnt_id = 'JOGT' GROUP BY prop
)



 Select reservation_date, market_name, ABW, media_desc, rs.mkt_location_des, gross_resv, canx_resv 
   , b.beach, h.hike, j.jog
   from RESERVATIONS rs
 inner join REVENUE rv on rs.conf_nbr = rv.conf_nbr
 left join CANCELS cn on rs.conf_nbr = cn.conf_nbr
 full outer join BEACH b on rs.prop_code = b.prop
 full outer join HIKE h on rs.prop_code = h.prop
 full outer join JOG j on rs.prop_code = j.prop 

 where rs.same_date between  '02-23-2020' and SYSDATE -2 AND reservation_date > '2020-02-23'

 ORDER BY reservation_date
"""

sql2 = sql2.replace("\n", "")

In [9]:
df = vertica_pull(sql2, False, False)
df.fillna(0)
df.to_csv("amenity_cancels.csv")

In [10]:
markets = " SELECT market_name, city, Geography, state  FROM prd_prop_info_detail GROUP BY 1,2,3,4 ORDER BY 1,2"
markets = vertica_pull(markets, False, True, "market_lookup")

File saved as market_lookup.csv


In [12]:
import psycopg2
con=psycopg2.connect("dbname=dap host=dap-redshift-proxy.prod.aws.chotel.com port=5439 user=danny_habetyes password==ire=og4Xotitrlfrog7")

In [13]:
cursor = con.cursor()

In [28]:
column_names = []
cursor.execute("select date(date_time), prop47, post_evar37,post_evar75, post_purchaseid, left(post_purchaseid, 8) as truncated_purchaseid, product_list, SPLIT_PART(product_list, ';', 4)  as revenue, curr_rate, SPLIT_PART(product_list, ';', 3) as nights from enterprise_omniture_event_local.omniture_website_hit_data where date(date_time)='2020-04-29' and post_purchaseid is not null and duplicate_purchase='0' and product_list is not null and SPLIT_PART(product_list, ';', 4)   ~  '^[0-9.]+$' and post_evar75 IS NOT NULL and post_purchaseid IS NOT NULL order by 1,2,3")
column_names = [desc[0] for desc in cursor.description]
all_cols=', '.join([str(x) for x in column_names])
print(all_cols)

date, prop47, post_evar37, post_evar75, post_purchaseid, truncated_purchaseid, product_list, revenue, curr_rate, nights


In [29]:
rs = pd.DataFrame(cursor.fetchall(), columns = [x[0] for x in cursor.description])

In [30]:
rs

,date,prop47,post_evar37,post_evar75,post_purchaseid,truncated_purchaseid,product_list,revenue,curr_rate,nights
0,2020-04-29,desktop,Google organic,EAIaIQobChMInofQiuWM6QIVD47ICh2prQ_kEAAYASAAEg...,49541784200429,49541784,QI;MS111;6;460.75,460.75,1.000000000000,6
1,2020-04-29,desktop,apps.choicecentral.com,EAIaIQobChMIlIe9iYfc4QIVx5CfCh2ctgZrEAAYASAAEg...,49524240200429,49524240,BR;FLC06;3;238.5,238.5,1.000000000000,3
2,2020-04-29,desktop,apps.choicecentral.com,EAIaIQobChMIlIe9iYfc4QIVx5CfCh2ctgZrEAAYASAAEg...,49524185200429,49524185,CS;FL952;2;70,70,1.000000000000,2
3,2020-04-29,desktop,apps.choicecentral.com,EAIaIQobChMI36W1v8aM6QIVDtvACh2kcwCaEAAYASAAEg...,49550402200429,49550402,SL;KS106;4;196,196,1.000000000000,4
4,2020-04-29,desktop,apps.choicecentral.com,EAIaIQobChMIlIe9iYfc4QIVx5CfCh2ctgZrEAAYASAAEg...,49524585200429,49524585,CS;FL952;2;70,70,1.000000000000,2
...,...,...,...,...,...,...,...,...,...,...
908,2020-04-29,is_tablet,mc:smgogouscin,Cj0KCQjwy6T1BRDXARIsAIqCTXq9VUi5NJPMxHfNeA1BNH...,49528593200429,49528593,CS;NM007;1;80.1,80.1,1.000000000000,1
909,2020-04-29,is_tablet,mc:smgogouscsn,EAIaIQobChMIhsv31JWO6QIVrgOzAB1ffAdWEAAYASAAEg...,49538667200429,49538667,CS;TXA48;2;170.98,170.98,1.000000000000,2
910,2020-04-29,is_tablet,www.bing.com,CICRtIL3jOkCFZyrxQIdIB8BSw,49519392200429,49519392,EL;TX270;1;47.97,47.97,1.000000000000,1
911,2020-04-29,is_tablet,None,Cj0KCQjwy6T1BRDXARIsAIqCTXpOo175wna2H0g0xZizGN...,4955671120429,49556711,CI;MD196;1;48,48,1.000000000000,1


In [27]:
cursor.fetchall()

[]